<a href="https://colab.research.google.com/github/dchlseo/DS-Toolkit/blob/master/hand_movement_yolo_new_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# BASIC CONFIG (필요한 파라미터 값 여기서 설정)

data_dir = '/content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/data/data_newLabels/'
model = 'yolov8n.pt'  # pretrained yolov8n

classes = ['0', '1', '2']  # 라벨명
nc = len(classes) # 라벨 수
yaml_file = 'hand_movement.yaml'  # yaml 파일명

training_epoch = 20
patience = 5  # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
batch = 16  # batch size (dafault)
imgsz = 640  # input batch size (default)
optimizer = 'Adam'


In [3]:
!nvidia-smi

Tue Dec  5 03:07:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### ULTRALYTICS SETUP

In [4]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.9/166.8 GB disk)


In [5]:
from ultralytics import YOLO
import os
import yaml
from IPython.display import display, Image
from IPython import display

display.clear_output()
!yolo checks

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.9/166.8 GB disk)

OS                  Linux-5.15.120+-x86_64-with-glibc2.35
Environment         Colab
Python              3.10.12
Install             pip
RAM                 51.00 GB
CPU                 Intel Xeon 2.20GHz
CUDA                11.8

matplotlib          ✅ 3.7.1>=3.3.0
numpy               ✅ 1.23.5>=1.22.2
opencv-python       ✅ 4.8.0.76>=4.6.0
pillow              ✅ 9.4.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.11.4>=1.4.1
torch               ✅ 2.1.0+cu118>=1.8.0
torchvision         ✅ 0.16.0+cu118>=0.9.0
tqdm                ✅ 4.66.1>=4.64.0
pandas              ✅ 1.5.3>=1.1.4
seaborn             ✅ 0.12.2>=0.11.0
psutil              ✅ 5.9.5
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1


In [ ]:
# # install comet
# %pip install comet_ml --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.4/586.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 28.6 MB/s eta 0:00:00


In [ ]:
# #@title Select YOLOv8 🚀 logger {run: 'auto'}
# logger = 'Comet' #@param ['Comet', 'TensorBoard']

# if logger == 'Comet':
#   %pip install -q comet_ml
#   import comet_ml; comet_ml.init()
# elif logger == 'TensorBoard':
#   %load_ext tensorboard
#   %tensorboard --logdir .

### Create yaml file

In [15]:
# python: dictionary <----------> yaml
data = {
    'train': data_dir + 'train',
    'val': data_dir + 'val',
    'test': data_dir + 'test',
    'nc': nc,
    'names': classes
}

os.makedirs(data_dir, exist_ok = True) # directory to save

with open(data_dir + yaml_file, 'wt') as fw:
    yaml.dump(data, fw)


In [16]:
with open(data_dir + yaml_file, 'rt') as fr:
    d = yaml.safe_load(fr)
    print(type(d))
    print(d)

<class 'dict'>
{'names': ['0', '1', '2'], 'nc': 3, 'test': '/content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/data/data_newLabels/test', 'train': '/content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/data/data_newLabels/train', 'val': '/content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/data/data_newLabels/val'}


### Model Training

In [17]:
model = YOLO(model)  # pretrained yolov8n

model.train(data = data_dir + yaml_file,
            epochs = training_epoch,
            patience = patience, # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
            batch = batch, # batch size (dafault)
            imgsz = imgsz, # input batch size (default)
            optimizer = optimizer)

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/data/data_newLabels/hand_movement.yaml, epochs=20, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning /content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/data/data_newLabels/train/labels... 490 images, 0 backgrounds, 0 corrupt: 100%|██████████| 490/490 [00:32<00:00, 14.90it/s]


train: New cache created: /content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/data/data_newLabels/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/data/data_newLabels/val/labels... 128 images, 0 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:07<00:00, 16.47it/s]

val: New cache created: /content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/data/data_newLabels/val/labels.cache


...

                   all        128        128       0.66      0.907      0.879        0.7



20 epochs completed in 0.027 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.42it/s]


                   all        128        128       0.66      0.903      0.878        0.7
                     0        128         35      0.582          1      0.881      0.717
                     1        128         53      0.746      0.774      0.844      0.673
                     2        128         40      0.652      0.935       0.91      0.711
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 7.9ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78df2a608ee0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

### Model Evaluation

In [19]:
best_model_path = '/content/drive/MyDrive/00DataScience/ds_study/DLproject/hand_movement/yolov8n_NL.pt'

best_model = YOLO(best_model_path)
metrics = best_model.val()

for label, ap in zip(classes, metrics.box.maps):
    print(label, ':', ap)

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

Dataset 'coco.yaml' images not found ⚠️, missing path '/content/datasets/coco/val2017.txt'


100%|██████████| 169M/169M [00:02<00:00, 75.9MB/s]
Unzipping /content/datasets/coco2017labels-segments.zip to /content/datasets/coco...: 100%|██████████| 122232/122232 [00:17<00:00, 7078.20file/s]


KeyboardInterrupt: ignored

In [ ]:
detect_path = '/content/runs/detect'

# download above folder to local pc


In [ ]:
# prompt: download folder in following directory to local pc: detect_path = '/content/runs/detect'

!zip -r detect_path.zip {detect_path}
!cp detect_path.zip /content/drive/MyDrive


NotImplementedError: ignored